✅1️⃣ **generate simple file inventories**  

In [ ]:
import yaml, os, json
from datetime import datetime

In [ ]:
inventory_dirs = "./"
config = "more-nosatsu.yaml"
# update config file name here ^^^

with open(os.path.join(inventory_dirs, config)) as yf:
    data = yaml.safe_load(yf)

In [ ]:
# check
for dir in data['dirs']:
    if data['top_level'] != None:
        location = os.path.join(data["top_level"], data['dirs'][dir])
    else:
        location = data['dirs'][dir]
    print(f"{location}: {os.listdir(location)}")
# end check

In [ ]:
for dir in data['dirs']:
    file_exts = []
    set = []
    if data['top_level'] != None:
        location = os.path.join(data["top_level"], data['dirs'][dir])
    else:
        location = data['dirs'][dir]
    report = {
        "date": datetime.now().strftime("%Y%m%d"),
        "path": location,
    }
    for file in os.listdir(location):
        if os.path.isfile(os.path.join(location, file)):
            set.append(file)
            if file.split('.')[-1] not in file_exts:
                file_exts.append(file.split('.')[-1])
    # report.update({"file_exts": file_exts}) # don't need this list in output, just need counts
    for file_ext in file_exts:
        counter = 0
        for file in os.listdir(location):
            if os.path.isfile(os.path.join(location, file)):
                if file.split('.')[-1] == file_ext:
                    counter += 1
        report.update({file_ext: counter})
    report.update({"total_file_count": len(set)})
    report.update({"set": set})
    # print(os.path.split(location)[1]) # check
    # print(json.dumps(report, indent=4)) # check
    filename = os.path.split(location)[1]
    with open(f"{location}/inventory_{filename}.yaml", mode="w") as yf:
        yaml.safe_dump(report, yf)


❌2️⃣ **compare files against simple file inventories** 

In [ ]:
# here's how I did it this one time, not very reusable
i_1805174 = "/home/nebgreb/desktop/inventory_1805174_NE1184_35_n67_v54.yaml"
i_19588069 = "/home/nebgreb/desktop/inventory_19588069_NE1184_35_N67_V54.yaml"
with open(i_1805174, "r") as yf:
    all_1805174 = yaml.safe_load(yf)
files_1805174 = all_1805174['set']
print(f"len(files_1805174): {len(files_1805174)}")
with open(i_19588069, 'r') as yf:
    all_19588069 = yaml.safe_load(yf)
files_19588069 = all_19588069['set']
print(f"len(files_19588069): {len(files_19588069)}")

In [ ]:
# set_180 = set(files_1805174)
# set_195 = set(files_19588069)
# only_180 = set_180 - set_195
# if only_180:
#     print("files only in 1805174 folder:")
#     for item in sorted(only_180):
#         print(item)
# only_195 = set_195 - set_180
# if only_195:
#     print("files only in 19588069 folder:")
#     for item in sorted(only_195):
#         print(item)

""" thought I could do set(list)...??
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Cell In[13], line 1
----> 1 set_180 = set(files_1805174)
      2 set_195 = set(files_19588069)
      3 only_180 = set_180 - set_195

TypeError: 'list' object is not callable
"""

In [ ]:
print("in 180... not in 195...")
for file in sorted(files_1805174):
    if file not in files_19588069:
        print(file)
print("in 195... not in 180...")
for file in sorted(files_19588069):
    if file not in files_1805174:
        print(file)

In [ ]:
print("in both!?")
for file in sorted(files_1805174):
    if file in files_19588069:
        print(file)

❌2️⃣🅰️ **visualize overlap in files between two dirs in two columns**  

In [ ]:
import re
filenames = os.listdir("/home/nebgreb/desktop/")
data = {}
p = re.compile(r'^inventory_(\d+)_.*$')
for filename in filenames:
    try:
        num = re.search(r'inventory_(\d+)_.*', filename).group(1)
        with open(os.path.join("/home/nebgreb/desktop/", filename), "r") as yf:
            inventory = yaml.safe_load(yf)
        data.update({num: inventory['set']})
    except:
        pass
print(data)
for item in data:
    print(len(data[item]))

In [ ]:
print(set(data['1805174']).union(set(data['19588069'])))
print(type(set(data['1805174']).union(set(data['19588069']))))
print(len(set(data['1805174']).union(set(data['19588069']))))
union_list = list(set(data['1805174']).union(set(data['19588069'])))
print(union_list)
print(type(union_list))
print(len(union_list))

In [ ]:
import csv
set_180 = data['1805174']
set_195 = data['19588069']
with open("/home/nebgreb/desktop/compare_v54_files.csv", "w") as csvf:
    writer = csv.writer(csvf)
    writer.writerow(["in-1805174", "in-19588069"])
    for item in sorted(union_list):
        row = []
        if item in set_180:
            row.append(item)
        else:
            row.append('')
        if item in set_195:
            row.append(item)
        else:
            row.append('')
        writer.writerow(row)